In [3]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score

In [4]:
dataset = pd.read_csv('spam_tfidf.csv', index_col=[0])
dataset.head()

,1,2,3,4,5,6,7,8,9,10,...,46,47,48,49,50,51,52,53,54,targhet
0,0.000000,1.04567,0.570084,0.0,0.309696,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.553776,0.000000,0.00000,1
1,0.309672,0.45748,0.445378,0.0,0.135492,0.427637,0.365548,0.120390,0.000000,1.186630,...,0.000000,0.0,0.0,0.00000,0.069628,0.0,0.264787,0.214164,0.08707,1
2,0.088478,0.00000,0.632436,0.0,1.190396,0.290182,0.330734,0.206383,1.141600,0.315593,...,0.131159,0.0,0.0,0.01762,0.075430,0.0,0.196455,0.218923,0.01814,1
3,0.000000,0.00000,0.000000,0.0,0.609715,0.000000,0.539619,1.083511,0.552962,0.795294,...,0.000000,0.0,0.0,0.00000,0.072265,0.0,0.097516,0.000000,0.00000,1
4,0.000000,0.00000,0.000000,0.0,0.609715,0.000000,0.539619,1.083511,0.552962,0.795294,...,0.000000,0.0,0.0,0.00000,0.071210,0.0,0.096092,0.000000,0.00000,1


In [5]:
from sklearn.model_selection import train_test_split
X = dataset.drop('targhet', axis=1)
y = dataset['targhet']  # colonna che segna se è spam o meno
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [39]:
import numpy as np

class GaussianNB:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.parameters = []
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.parameters.append([])
            for col in X_c.T:
                parameters = {"mean": np.array(col).mean(), "var": np.array(col).var()}
                self.parameters[i].append(parameters)

    def predict(self, X):
        y_pred = []
        for sample in X:
            posteriors = []
            for i, c in enumerate(self.classes):
                likelihood = 1
                for feature, params in zip(sample, self.parameters[i]):
                    likelihood *= self._pdf(params["mean"], params["var"], feature)
                prior = len(self.X_c) / len(X)
                posterior = likelihood * prior
                posteriors.append(posterior)
            y_pred.append(self.classes[np.argmax(posteriors)])
        return y_pred

    def _pdf(self, mean, var, x):
        eps = 1e-4 # added in case var == 0
        coef = 1.0 / np.sqrt(2.0 * np.pi * var + eps)
        exponent = -((x - mean)**2 / (2 * var + eps))
        return coef * np.exp(exponent)


In [40]:
# from sklearn.naive_bayes import GaussianNB
nbg = GaussianNB()
start_time=time.time()
nbg.fit(X_train, y_train)
print('Training time: %f'%(time.time() - start_time))
start_time=time.time()
y_pred_nbb = nbg.predict(X_test)
print('Prediction time: %f'%(time.time() - start_time))
print('Missclassified examples: %d'% (y_test != y_pred_nbb).sum())
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred_nbb))

Training time: 0.164568


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U1'), dtype('float64')) -> None